In [85]:
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from sklearn.neighbors import NearestNeighbors

In [3]:
df=pd.read_csv('yelp_coffeeshop_review_data.csv')

In [4]:
df.head()

,coffee_shop_name,full_review_text,star_rating
0,The Factory - Cafe With a Soul,11/25/2016 1 check-in Love love loved the atm...,5.0 star rating
1,The Factory - Cafe With a Soul,"12/2/2016 Listed in Date Night: Austin, Ambia...",4.0 star rating
2,The Factory - Cafe With a Soul,11/30/2016 1 check-in Listed in Brunch Spots ...,4.0 star rating
3,The Factory - Cafe With a Soul,11/25/2016 Very cool decor! Good drinks Nice ...,2.0 star rating
4,The Factory - Cafe With a Soul,12/3/2016 1 check-in They are located within ...,4.0 star rating


In [11]:
stopWords=list(STOPWORDS)
def tokenize(data):
    tokens=[]
    for token in simple_preprocess(data):
        if token not in stopWords:
            tokens.append(token)
    del tokens[0]
    try:
        int(tokens[0])
        del tokens[:1]
    except:
        pass
    return tokens

df['tokens']=df['full_review_text'].apply(tokenize)
df['stars']=df['star_rating'].apply(lambda x: float(x.split()[0]))

In [16]:
df.describe(exclude='number')

,coffee_shop_name,full_review_text,star_rating,tokens
count,7616,7616,7616,7616
unique,79,6915,5,6915
top,Epoch Coffee,10/15/2016 This place is really nice. They ha...,5.0 star rating,"[listed, search, fantastic, coffee, want, writ..."
freq,400,4,3780,4


In [20]:
shops=df.coffee_shop_name.value_counts()

In [24]:
shops[shops>99].shape

(58,)

In [26]:
tfidf=TfidfVectorizer(stop_words=stopWords)

In [28]:
dtm=tfidf.fit_transform(df['tokens'].apply(lambda x: ' '.join(x)))

In [86]:
tfiDF=pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

In [79]:
id2word=corpora.Dictionary(df['tokens'])
len(id2word.keys())

16468

In [80]:
allWords=[id2word[l] for l in id2word.keys()]

In [81]:
id2word.filter_extremes(no_above=10)

In [82]:
len(id2word.keys())

5328

In [83]:
goWords=[id2word[l] for l in id2word.keys()]

In [76]:
stops=allWords.remove(goWords)

ValueError: list.remove(x): x not in list

In [87]:
nn=NearestNeighbors(n_neighbors=5, algorithm='kd_tree')

In [90]:
tfiDF['ShopName']=df.coffee_shop_name
shops=tfiDF.groupby('ShopName').mean()

In [91]:
nn.fit(shops)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [101]:
shopNames=list(shops.index)

In [104]:
nn.kneighbors([shops.loc['Cream Whiskers ']])

(array([[0.        , 0.41443899, 0.42157103, 0.42399332, 0.42418638]]),
 array([[14, 21, 54, 70, 17]]))

In [106]:
shopNames[21]

'Fat Cats Organic Coffee And Dessert '